# import 과정

한글 폰트 받기
코드 실행 후 런타임 재시작하여 다시 실행

In [ ]:
!apt -qq -y install fonts-nanum > /dev/null # 폰트 나눔을 구글코랩에 설치하고 경로를 파악함
import matplotlib.font_manager as fm 
fontpath='/user/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font=fm.FontProperties(fname=fontpath,size=9)

In [ ]:
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings(action='ignore') 

plt.rc('font', family='NanumBarunGothic') 

STT - Whisper

In [ ]:
!pip install git+https://github.com/openai/whisper.git
!sudo apt update && sudo apt install ffmpeg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-tpbfttxx
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-tpbfttxx
  Resolved https://github.com/openai/whisper.git to commit 248b6cb124225dd263bb9bd32d060b6517e067f8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 24.3 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20230314-py3-none-any.whl size=798075 sha256=f5a6b93aacfc837496d2efab233c50d0e118d333e4be8c96678573be030a647c
  Stored in directory: /tmp/pip-ephem-wheel-cache-_tdflanb/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-whisper
Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-

In [ ]:
#!whisper 'audio1.wav' --model medium

In [ ]:
import whisper
model = whisper.load_model("medium")

100%|██████████████████████████████████████| 1.42G/1.42G [00:11<00:00, 138MiB/s]


# Gaze Tracking

In [ ]:
!git clone https://github.com/antoinelame/GazeTracking.git

Cloning into 'GazeTracking'...
remote: Enumerating objects: 113, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 113 (delta 34), reused 30 (delta 30), pack-reused 67
Receiving objects: 100% (113/113), 69.00 MiB | 31.63 MiB/s, done.
Resolving deltas: 100% (56/56), done.


In [ ]:
import numpy as np
import cv2


class Pupil(object):
    """
    This class detects the iris of an eye and estimates
    the position of the pupil
    """

    def __init__(self, eye_frame, threshold):
        self.iris_frame = None
        self.threshold = threshold
        self.x = None
        self.y = None

        self.detect_iris(eye_frame)

    @staticmethod
    def image_processing(eye_frame, threshold):
        """Performs operations on the eye frame to isolate the iris
        Arguments:
            eye_frame (numpy.ndarray): Frame containing an eye and nothing else
            threshold (int): Threshold value used to binarize the eye frame
        Returns:
            A frame with a single element representing the iris
        """
        kernel = np.ones((3, 3), np.uint8)
        new_frame = cv2.bilateralFilter(eye_frame, 10, 15, 15)
        new_frame = cv2.erode(new_frame, kernel, iterations=3)
        new_frame = cv2.threshold(new_frame, threshold, 255, cv2.THRESH_BINARY)[1]

        return new_frame

    def detect_iris(self, eye_frame):
        """Detects the iris and estimates the position of the iris by
        calculating the centroid.
        Arguments:
            eye_frame (numpy.ndarray): Frame containing an eye and nothing else
        """
        self.iris_frame = self.image_processing(eye_frame, self.threshold)

        contours, _ = cv2.findContours(self.iris_frame, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)[-2:]
        contours = sorted(contours, key=cv2.contourArea)

        try:
            moments = cv2.moments(contours[-2])
            self.x = int(moments['m10'] / moments['m00'])
            self.y = int(moments['m01'] / moments['m00'])
        except (IndexError, ZeroDivisionError):
            pass

In [ ]:
from __future__ import division
import cv2
#from .pupil import Pupil


class Calibration(object):
    """
    This class calibrates the pupil detection algorithm by finding the
    best binarization threshold value for the person and the webcam.
    """

    def __init__(self):
        self.nb_frames = 20
        self.thresholds_left = []
        self.thresholds_right = []

    def is_complete(self):
        """Returns true if the calibration is completed"""
        return len(self.thresholds_left) >= self.nb_frames and len(self.thresholds_right) >= self.nb_frames

    def threshold(self, side):
        """Returns the threshold value for the given eye.
        Argument:
            side: Indicates whether it's the left eye (0) or the right eye (1)
        """
        if side == 0:
            return int(sum(self.thresholds_left) / len(self.thresholds_left))
        elif side == 1:
            return int(sum(self.thresholds_right) / len(self.thresholds_right))

    @staticmethod
    def iris_size(frame):
        """Returns the percentage of space that the iris takes up on
        the surface of the eye.
        Argument:
            frame (numpy.ndarray): Binarized iris frame
        """
        frame = frame[5:-5, 5:-5]
        height, width = frame.shape[:2]
        nb_pixels = height * width
        nb_blacks = nb_pixels - cv2.countNonZero(frame)
        return nb_blacks / nb_pixels

    @staticmethod
    def find_best_threshold(eye_frame):
        """Calculates the optimal threshold to binarize the
        frame for the given eye.
        Argument:
            eye_frame (numpy.ndarray): Frame of the eye to be analyzed
        """
        average_iris_size = 0.48
        trials = {}

        for threshold in range(5, 100, 5):
            iris_frame = Pupil.image_processing(eye_frame, threshold)
            trials[threshold] = Calibration.iris_size(iris_frame)

        best_threshold, iris_size = min(trials.items(), key=(lambda p: abs(p[1] - average_iris_size)))
        return best_threshold

    def evaluate(self, eye_frame, side):
        """Improves calibration by taking into consideration the
        given image.
        Arguments:
            eye_frame (numpy.ndarray): Frame of the eye
            side: Indicates whether it's the left eye (0) or the right eye (1)
        """
        threshold = self.find_best_threshold(eye_frame)

        if side == 0:
            self.thresholds_left.append(threshold)
        elif side == 1:
            self.thresholds_right.append(threshold)

In [ ]:
import math
import numpy as np
import cv2
#from .pupil import Pupil


class Eye(object):
    """
    This class creates a new frame to isolate the eye and
    initiates the pupil detection.
    """

    LEFT_EYE_POINTS = [36, 37, 38, 39, 40, 41]
    RIGHT_EYE_POINTS = [42, 43, 44, 45, 46, 47]

    def __init__(self, original_frame, landmarks, side, calibration):
        self.frame = None
        self.origin = None
        self.center = None
        self.pupil = None
        self.landmark_points = None

        self._analyze(original_frame, landmarks, side, calibration)

    @staticmethod
    def _middle_point(p1, p2):
        """Returns the middle point (x,y) between two points

        Arguments:
            p1 (dlib.point): First point
            p2 (dlib.point): Second point
        """
        x = int((p1.x + p2.x) / 2)
        y = int((p1.y + p2.y) / 2)
        return (x, y)

    def _isolate(self, frame, landmarks, points):
        """Isolate an eye, to have a frame without other part of the face.

        Arguments:
            frame (numpy.ndarray): Frame containing the face
            landmarks (dlib.full_object_detection): Facial landmarks for the face region
            points (list): Points of an eye (from the 68 Multi-PIE landmarks)
        """
        region = np.array([(landmarks.part(point).x, landmarks.part(point).y) for point in points])
        region = region.astype(np.int32)
        self.landmark_points = region

        # Applying a mask to get only the eye
        height, width = frame.shape[:2]
        black_frame = np.zeros((height, width), np.uint8)
        mask = np.full((height, width), 255, np.uint8)
        cv2.fillPoly(mask, [region], (0, 0, 0))
        eye = cv2.bitwise_not(black_frame, frame.copy(), mask=mask)

        # Cropping on the eye
        margin = 5
        min_x = np.min(region[:, 0]) - margin
        max_x = np.max(region[:, 0]) + margin
        min_y = np.min(region[:, 1]) - margin
        max_y = np.max(region[:, 1]) + margin

        self.frame = eye[min_y:max_y, min_x:max_x]
        self.origin = (min_x, min_y)

        height, width = self.frame.shape[:2]
        self.center = (width / 2, height / 2)

    def _blinking_ratio(self, landmarks, points):
        """Calculates a ratio that can indicate whether an eye is closed or not.
        It's the division of the width of the eye, by its height.

        Arguments:
            landmarks (dlib.full_object_detection): Facial landmarks for the face region
            points (list): Points of an eye (from the 68 Multi-PIE landmarks)

        Returns:
            The computed ratio
        """
        left = (landmarks.part(points[0]).x, landmarks.part(points[0]).y)
        right = (landmarks.part(points[3]).x, landmarks.part(points[3]).y)
        top = self._middle_point(landmarks.part(points[1]), landmarks.part(points[2]))
        bottom = self._middle_point(landmarks.part(points[5]), landmarks.part(points[4]))

        eye_width = math.hypot((left[0] - right[0]), (left[1] - right[1]))
        eye_height = math.hypot((top[0] - bottom[0]), (top[1] - bottom[1]))

        try:
            ratio = eye_width / eye_height
        except ZeroDivisionError:
            ratio = None

        return ratio

    def _analyze(self, original_frame, landmarks, side, calibration):
        """Detects and isolates the eye in a new frame, sends data to the calibration
        and initializes Pupil object.

        Arguments:
            original_frame (numpy.ndarray): Frame passed by the user
            landmarks (dlib.full_object_detection): Facial landmarks for the face region
            side: Indicates whether it's the left eye (0) or the right eye (1)
            calibration (calibration.Calibration): Manages the binarization threshold value
        """
        if side == 0:
            points = self.LEFT_EYE_POINTS
        elif side == 1:
            points = self.RIGHT_EYE_POINTS
        else:
            return

        self.blinking = self._blinking_ratio(landmarks, points)
        self._isolate(original_frame, landmarks, points)

        if not calibration.is_complete():
            calibration.evaluate(self.frame, side)

        threshold = calibration.threshold(side)
        self.pupil = Pupil(self.frame, threshold)

In [ ]:
from __future__ import division
import os
import cv2
import dlib
#from .eye import Eye
#from .calibration import Calibration


class GazeTracking(object):
    """
    This class tracks the user's gaze.
    It provides useful information like the position of the eyes
    and pupils and allows to know if the eyes are open or closed
    """

    def __init__(self):
        self.frame = None
        self.eye_left = None
        self.eye_right = None
        self.calibration = Calibration()

        # _face_detector is used to detect faces
        self._face_detector = dlib.get_frontal_face_detector()

        # _predictor is used to get facial landmarks of a given face
        #cwd = os.path.abspath(os.path.dirname(__file__))
        #model_path = os.path.abspath(os.path.join(cwd, "trained_models/shape_predictor_68_face_landmarks.dat"))
        #self._predictor = dlib.shape_predictor(model_path)
        self._predictor = dlib.shape_predictor('/content/GazeTracking/gaze_tracking/trained_models/shape_predictor_68_face_landmarks.dat')

    @property
    def pupils_located(self):
        """Check that the pupils have been located"""
        try:
            int(self.eye_left.pupil.x)
            int(self.eye_left.pupil.y)
            int(self.eye_right.pupil.x)
            int(self.eye_right.pupil.y)
            return True
        except Exception:
            return False

    def _analyze(self):
        """Detects the face and initialize Eye objects"""
        frame = cv2.cvtColor(self.frame, cv2.COLOR_BGR2GRAY)
        faces = self._face_detector(frame)

        try:
            landmarks = self._predictor(frame, faces[0])
            self.eye_left = Eye(frame, landmarks, 0, self.calibration)
            self.eye_right = Eye(frame, landmarks, 1, self.calibration)

        except IndexError:
            self.eye_left = None
            self.eye_right = None

    def refresh(self, frame):
        """Refreshes the frame and analyzes it.
        Arguments:
            frame (numpy.ndarray): The frame to analyze
        """
        self.frame = frame
        self._analyze()

    def pupil_left_coords(self):
        """Returns the coordinates of the left pupil"""
        if self.pupils_located:
            x = self.eye_left.origin[0] + self.eye_left.pupil.x
            y = self.eye_left.origin[1] + self.eye_left.pupil.y
            return (x, y)

    def pupil_right_coords(self):
        """Returns the coordinates of the right pupil"""
        if self.pupils_located:
            x = self.eye_right.origin[0] + self.eye_right.pupil.x
            y = self.eye_right.origin[1] + self.eye_right.pupil.y
            return (x, y)

    def horizontal_ratio(self):
        """Returns a number between 0.0 and 1.0 that indicates the
        horizontal direction of the gaze. The extreme right is 0.0,
        the center is 0.5 and the extreme left is 1.0
        """
        if self.pupils_located:
            pupil_left = self.eye_left.pupil.x / (self.eye_left.center[0] * 2 - 10)
            pupil_right = self.eye_right.pupil.x / (self.eye_right.center[0] * 2 - 10)
            return (pupil_left + pupil_right) / 2

    def vertical_ratio(self):
        """Returns a number between 0.0 and 1.0 that indicates the
        vertical direction of the gaze. The extreme top is 0.0,
        the center is 0.5 and the extreme bottom is 1.0
        """
        if self.pupils_located:
            pupil_left = self.eye_left.pupil.y / (self.eye_left.center[1] * 2 - 10)
            pupil_right = self.eye_right.pupil.y / (self.eye_right.center[1] * 2 - 10)
            return (pupil_left + pupil_right) / 2

    def is_right(self):
        """Returns true if the user is looking to the right"""
        if self.pupils_located:
            return self.horizontal_ratio() <= 0.35

    def is_left(self):
        """Returns true if the user is looking to the left"""
        if self.pupils_located:
            return self.horizontal_ratio() >= 0.65

    def is_center(self):
        """Returns true if the user is looking to the center"""
        if self.pupils_located:
            return self.is_right() is not True and self.is_left() is not True

    def is_blinking(self):
        """Returns true if the user closes his eyes"""
        if self.pupils_located:
            blinking_ratio = (self.eye_left.blinking + self.eye_right.blinking) / 2
            return blinking_ratio > 3.8

    def annotated_frame(self):
        """Returns the main frame with pupils highlighted"""
        frame = self.frame.copy()

        if self.pupils_located:
            color = (0, 255, 0)
            x_left, y_left = self.pupil_left_coords()
            x_right, y_right = self.pupil_right_coords()
            cv2.line(frame, (x_left - 5, y_left), (x_left + 5, y_left), color)
            cv2.line(frame, (x_left, y_left - 5), (x_left, y_left + 5), color)
            cv2.line(frame, (x_right - 5, y_right), (x_right + 5, y_right), color)
            cv2.line(frame, (x_right, y_right - 5), (x_right, y_right + 5), color)

        return frame


In [ ]:
gaze = GazeTracking()

Konlpy

In [ ]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 69.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 40.8 MB/s eta 0:00:00


In [ ]:
from konlpy.tag import Okt
from konlpy.utils import pprint

In [ ]:
from collections import Counter
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import numpy as np

# 최종 코드

In [ ]:
!pip install ffmpeg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6083 sha256=926f3b9f13109a16643b0b6926d01202b32ba1b046911ca4bbb926c4f54566c3
  Stored in directory: /root/.cache/pip/wheels/8e/7a/69/cd6aeb83b126a7f04cbe7c9d929028dc52a6e7d525ff56003a
Successfully built ffmpeg


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
video_path = ' '

In [ ]:
import subprocess
import os

video_path # mp4 file
wav_path = "/content"             # 저장할 path
save_file_id = "/content/ .wav" # 저장할 file name 

command = "ffmpeg -i {} -ab 160k -ac 2 -ar 44100 -vn {}".format(video_path, os.path.join(wav_path, save_file_id))    
subprocess.call(command, shell=True)

1

In [ ]:
import librosa

In [ ]:
# 특정 표현이 사용된 횟수 출력

def count_words(word_list):
    word_count = {}
    for word in word_list:
        if word in word_count:
            word_count[word] += 1
        else:
            word_count[word] = 1
    return word_count

In [ ]:
# 애매한 표현, 말더듬기 출력

def speech_analyze(text):
  unclear = ['그냥', '아마', '아마도', '대충', '좀', '약간']
  umm = ['어', '음', '아', '그']
  unclear_list = []
  umm_list = []
  unclear_num = 0
  umm_num = 0

  punctuation = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
  # 문장부호 삭제
  result = text.translate(str.maketrans('', '', punctuation))

  words = result.split(" ")
  word_count = len(words)

  for i in range(word_count):
    for j in range(len(unclear)):
      if words[i]==unclear[j]: 
        unclear_num+=1
        unclear_list.append(words[i])
    for k in range(len(umm)):
      if words[i]==umm[k]:
        umm_num+=1
        umm_list.append(words[i])

  print(unclear_num, unclear_list, unclear_num/word_count)
  print(umm_num, umm_list, umm_num/word_count)

  unclear_count = count_words(unclear_list)
  umm_count = count_words(umm_list)

  for word, count in unclear_count.items():
    print(f'{word}: {count}')
    
  for word, count in umm_count.items():
    print(f'{word}: {count}')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
# 문장 단위 말하는 속도 그래프 출력

def speech_speed(result):
  remove_set = {' ', '.', ',', '?', '!' }
  word_rate = []

  for i in range(len(result['segments'])):
    text_set = result['segments'][i]
    start = result['segments'][i]['start']
    end = result['segments'][i]['end']

    text_list = list(text_set)
    text_list = [j for j in text_list if j not in remove_set]

    word_rate.append(len(text_list)/(end-start))


  print(word_rate)

  x = []
  for i in range(len(result['segments'])):
    x.append(result['segments'][i]['start'])
  y = [j for j in word_rate]
  print(np.nanmean(word_rate))

  #plt.subplot(2, 1, 2)  
  plt.axhline(y=np.nanmean(word_rate))
  plt.plot(x, y) 
  plt.xlabel("x") 
  plt.ylabel("y")

  return word_rate

In [ ]:
# stt

def speech_blink(result, video_path):
  # stt 워드 클라우드
  text = result["text"]

  okt = Okt()
  nouns = okt.nouns(text) # 명사만 추출

  words = [n for n in nouns if len(n) > 1] # 단어의 길이가 1개인 것은 제외

  c = Counter(words) 

  wc = WordCloud(font_path='user/share/fonts/truetype/nanum/NanumBarunGothic.ttf', width=400, height=400, scale=2.0, max_font_size=250)
  gen = wc.generate_from_frequencies(c)
  plt.figure()
  plt.imshow(gen)

  # 음성 빠르기


  # 텍스트에서 단어 수 카운트
  words_2 = text.split()
  word_count = len(words_2)

  # 음성 파일 길이 계산
  audio_file = save_file_id
  duration = librosa.get_duration(filename=audio_file)

  # spm 계산
  spm = word_count / duration

  # 결과 출력
  print("Word Count: {}".format(word_count))
  print("Duration: {:.2f} seconds".format(duration))
  print("Speech Rate: {:.2f} words per second".format(spm))
  if spm>=2.0: print("말하는 속도가 너무 빠릅니다. 속도 그래프를 참고해 적절한 말하기 속도를 찾아보세요.")

  #  Gaze tracking

  import cv2
  #from gaze_tracking import GazeTracking

  gaze = GazeTracking()
  video = cv2.VideoCapture(video_path) # 비디오 파일 경로 입력
  frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT)) # 비디오 프레임 수 확인

  blink_count = 0
  gaze_movement_count = 0

  while True:
    ret, frame = video.read()
    if not ret:
        break

    gaze.refresh(frame)
    frame = gaze.annotated_frame()

    if gaze.is_blinking():
        blink_count += 1

    if gaze.pupil_left_coords() and gaze.pupil_right_coords():
        gaze_movement_count += 1

  blink_rate = blink_count / frame_count # 눈 깜빡임 비율
  gaze_movement_rate = gaze_movement_count / frame_count # 눈동자 움직임 비율

  print(f"Blink rate: {blink_rate}")
  if blink_rate>=0.13: print("눈을 너무 많이 깜빡입니다.")
  print(f"Gaze movement rate: {gaze_movement_rate}")

  speech_analyze(text)


  #word_rate = speech_speed(result)



In [ ]:
result = model.transcribe(save_file_id)
print(result["text"])

In [ ]:
speech_blink(result)

In [ ]:
speech_speed(result)